In [ ]:
import numpy as np
import torch
from torch import optim
from torch.autograd import Variable

from models_code.vae import create_latent_space_results_is
from models_code.vae import create_latent_space_results_mc
from models_code.vae import plot_latent_space
from models_code.vae import show_decoded_latent_space
from models_code.vae import show_decoder_quality
from models_code.vae import test_autoencoder
from models_code.vae import train_autoencoder
from models_code.vae import VAE
from models_code.mnist import load_data

In [ ]:

batch_size = 128
log_interval = 100
epochs = 50

torch.manual_seed(9)
torch.cuda.manual_seed(9)


kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader, test_loader = load_data(batch_size)

model = VAE()
model.cuda()
optimizer = optim.Adam(model.parameters())


In [ ]:

model.load_state_dict(torch.load('models/vae.torch'))

# for epoch in range(1, epochs + 1):
#     train_autoencoder(epoch, train_loader, log_interval, model, optimizer)
#     test_autoencoder(test_loader, model)
    
# torch.save(model.state_dict(), 'models/vae.torch')


# Check examples

### This visualization will show that the autoencoder works, and the decoded images resemble the original

In [ ]:
_, (exs, _) = next(enumerate(test_loader))
mini_batch = exs[:25,:]

show_decoder_quality(mini_batch, model)

# First plot

### Firstly, create meshgrid and compute metrics

In [ ]:
from models_code.utilities import load_model
from models_code.mnist import MCMnist
from models_code.mnist import ISMnist
import math

In [ ]:
modelmc = load_model(MCMnist, 'models/mnist_lenet/mc.torch')
modelis = load_model(ISMnist, 'models/mnist_lenet/is.torch')

In [ ]:
p_ents, e_ents, m_info, latent_X, latent_Y, decoded_imgs = create_latent_space_results_mc(-10, 10, modelmc, model)

In [ ]:
is_conv_confidence, latent_X, latent_Y, decoded_imgs = create_latent_space_results_is(-10, 10, modelis, model)

### Then, create plots

In [ ]:
# Inference for all training examples

xses = []
logvars = []
yses = []
images = []

for batch_idx, (data, ys) in enumerate(train_loader):
    model.eval()
    data = Variable(data)
    data = data.cuda()
    recon_batch, mu, logvar = model(data)

    xses.append(mu.cpu().data.numpy())
    yses.append(Variable(ys).cpu().data.numpy())
    logvars.append(logvar.cpu().data.numpy())
    
xses = np.concatenate(xses)
yses = np.concatenate(yses)


In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 40})

from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from matplotlib import pyplot as plt

In [ ]:
cccv = CalibratedClassifierCV(cv='prefit')

In [ ]:
def calibrate(array2d):
    '''Changes the probability density of latent space certainities into uniform'''
    thresholds = np.sort(array2d.flatten())[::100]
    return np.digitize(array2d, thresholds)

In [ ]:
# p_ents = calibrate(p_ents)
is_conv_confidence = calibrate(is_conv_confidence)

In [ ]:
plot_latent_space(
    latent_X,
    latent_Y,
    is_conv_confidence,
    xses,
    yses,
    "",
    norm=None,
    savepath='images/vae_is.png'
)

In [ ]:

plot_latent_space(
    latent_X[60:140, 60:140],
    latent_Y[60:140, 60:140],
    is_conv_confidence[60:140, 60:140],
    xses[(np.abs(xses[:,0]) < 4) & (np.abs(xses[:,1]) < 4)],
    yses[(np.abs(xses[:,0]) < 4) & (np.abs(xses[:,1]) < 4)],
    "",
    norm=None,
    savepath='images/vae_is_small.png'
)

# Plots for MC Dropout

In [ ]:
plot_latent_space(
    latent_X,
    latent_Y,
    p_ents,
    xses,
    yses,
    "",
    norm=None,
    savepath='images/vae_mc.png'
)

In [ ]:
plot_latent_space(
    latent_X[60:140, 60:140],
    latent_Y[60:140, 60:140],
    p_ents[60:140, 60:140],
    xses[(np.abs(xses[:,0]) < 4) & (np.abs(xses[:,1]) < 4)],
    yses[(np.abs(xses[:,0]) < 4) & (np.abs(xses[:,1]) < 4)],
    "",
    norm=None,
    savepath='images/vae_mc_small.png'
)

# Show what latent space represents

In [ ]:
show_decoded_latent_space(model)